Baseline Algorithm v1: Direct Search Clasification: DiSeCla

Descripción detallada del proceso:

Importación de bibliotecas: Se importan las bibliotecas necesarias: pandas para el manejo de datos, nltk para la tokenización de texto y re para manipulación de cadenas.

Cargar archivos: Se cargan los archivos CSV ("HS Code catalogue.csv" y "Validation Data.csv") en DataFrames.

Definir funciones de limpieza y tokenización: clean_text(): limpia el texto de números y signos de puntuación, y lo convierte a minúsculas.

tokenize(): divide el texto en palabras individuales (tokens).

Preprocesar el catálogo de códigos HS: Se limpian y tokenizan las descripciones del catálogo de códigos HS.

Iterar sobre los datos de validación: Se procesan las primeras 20 filas de los datos de validación, limpiando y tokenizando las entradas.

Contar coincidencias: Para cada token en la entrada de validación, se comparan con las descripciones del catálogo de códigos HS y se cuentan las coincidencias.

Determinar el mejor código HS: Se selecciona el código HS con el mayor número de coincidencias basadas en una operación de igualdad y se guarda junto con la descripción y el conteo.

Guardar resultados: Los resultados se almacenan en una lista, se convierten en un nuevo DataFrame y se guardan en un archivo CSV.

Mostrar resultados: Se imprime un vistazo de los primeros 20 registros del DataFrame resultante.


In [2]:
# Importar las bibliotecas necesarias
import pandas as pd  # Para manejar los datos en formato de DataFrame
import nltk  # Para la tokenización y procesamiento de texto
import re  # Para limpiar y manipular texto con expresiones regulares
from nltk.tokenize import word_tokenize  # Función para tokenizar el texto
nltk.download('punkt')  # Descargar el paquete necesario para la tokenización

# Cargar los archivos CSV en DataFrames
hs_code_catalogue_df = pd.read_csv('../data/WebScrap_CSVs\hts_codes_WebScrapped.csv')
validation_data_df = pd.read_csv('../../Validation Data.csv')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
import re
import random
import pandas as pd
from nltk.tokenize import word_tokenize

# Function to clean the text (removes numbers and punctuation)
def clean_text(text):
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.lower()  # Convert text to lowercase

# Function to tokenize the text
def tokenize(text):
    return word_tokenize(text)  # Tokenize the text into individual words

# Preprocess the HS code catalogue descriptions
def preprocess_catalogue(hs_code_catalogue_df):
    hs_code_catalogue_df['cleaned_description'] = hs_code_catalogue_df['Description'].apply(clean_text)  # Clean text
    hs_code_catalogue_df['tokens'] = hs_code_catalogue_df['cleaned_description'].apply(tokenize)  # Tokenize descriptions
    return hs_code_catalogue_df

# Function to run one iteration and return the result DataFrame
def run_iteration(hs_code_catalogue_df, validation_data_df):
    # Create a list to store the results of validation
    output_data = []

    # Select 20 random rows from validation_data_df
    random_indices = random.sample(range(len(validation_data_df)), min(20, len(validation_data_df)))

    # Iterate over the randomly selected entries
    for i in random_indices:
        raw_input = validation_data_df['Raw_data_input'].iloc[i]
        cleaned_text = clean_text(raw_input)
        tokens = tokenize(cleaned_text)

        match_counts = {}

        # Compare tokens with HS code descriptions
        for token in tokens:
            for j, row in hs_code_catalogue_df.iterrows():
                if token in row['tokens']:
                    if row['HTS code'] not in match_counts:
                        match_counts[row['HTS code']] = 0
                    match_counts[row['HTS code']] += 1

        if match_counts:
            best_hs_code = max(match_counts, key=match_counts.get)
            best_hs_code_description = hs_code_catalogue_df.loc[hs_code_catalogue_df['HTS code'] == best_hs_code, 'Description'].values[0]
        else:
            best_hs_code = None
            best_hs_code_description = None

        hs_actual_from_validation = validation_data_df['Expected_output'].iloc[i]

        # Add the results to the output list
        output_data.append([
            raw_input,
            ' '.join(tokens),
            best_hs_code,
            best_hs_code_description,
            hs_actual_from_validation
        ])

    # Convert to DataFrame
    output_df = pd.DataFrame(output_data, columns=[
        'Raw_data_input',
        'Tokens',
        'Best_HS_Code',
        'Best_HS_Code_Description',
        'HS_Actual_from_validation'
    ])
    
    return output_df

# Example usage: Preprocess catalogue and run one iteration
hs_code_catalogue_df = preprocess_catalogue(hs_code_catalogue_df)
output_df = run_iteration(hs_code_catalogue_df, validation_data_df)

# Display first 10 rows of results
print(output_df.head(10))



                                      Raw_data_input  \
0  HOSTACOM G2 R03 105555 10 PALFREIGHT PREPAIDBA...   
1     COUPLINGS HS CODE 853669 PICK UP REF 515478325   
2  EN 23 PALLETS EMBUTIDOHS CODE 160100FREIGHT PR...   
3  WITH 40 BIG BAGS OF FHI990091A HS CODE 3904 ST...   
4  FRY PANHS CODE   200410COOKWARE SET  CASEROLE ...   
5  SETPOINT PLUS 18 DEGREES CELSIUS CONTRACT QANT...   
6  GEAR HOUSING INVOICENO1000007332 HS CODE 76169910   
7  1X20TANK SAID TO CONTAIN 1 ISO SOC TANK CHEMIC...   
8  PLATINA SOLDADA HS CODE 73269098 FREIGHT PREPA...   
9     CAR PARTS HS CODE 830230DN 76141621IN 86147476   

                                              Tokens  Best_HS_Code  \
0  hostacom g r palfreight prepaidbags loaded ont...       3808.62   
1                      couplings hs code pick up ref       7412.10   
2  en pallets embutidohs code freight prepaid thi...       9903.52   
3             with big bags of fhia hs code status n       4202.91   
4  fry panhs code cookware set ca

In [19]:
output_df.head()

,Raw_data_input,Tokens,Best_HS_Code,Best_HS_Code_Description,HS_Actual_from_validation
0,HOSTACOM G2 R03 105555 10 PALFREIGHT PREPAIDBA...,hostacom g r palfreight prepaidbags loaded ont...,3808.62,"Goods of heading 3808, containing alpha-cyperm...",39021000
1,COUPLINGS HS CODE 853669 PICK UP REF 515478325,couplings hs code pick up ref,7412.10,"Refined copper tube or pipe fittings ""e.g., co...",853669
2,EN 23 PALLETS EMBUTIDOHS CODE 160100FREIGHT PR...,en pallets embutidohs code freight prepaid thi...,9903.52,Notwithstanding any other quantitative limitat...,160100
3,WITH 40 BIG BAGS OF FHI990091A HS CODE 3904 ST...,with big bags of fhia hs code status n,4202.91,"Travelling-bags, insulated food or beverage ba...",3904
4,FRY PANHS CODE 200410COOKWARE SET CASEROLE ...,fry panhs code cookware set caserole fry panhs...,3002.19,"Immunological products, n.e.s. (code possibly ...",200410


In [18]:
import pandas as pd
import matplotlib.pyplot as plt

# Function to calculate accuracy (both for 2 and 4 digits)
def calculate_accuracy(output_df):
    correct_two_digits = 0
    correct_four_digits = 0
    total = len(output_df)

    for index, row in output_df.iterrows():
        best_hs_code = str(row['Best_HS_Code'])[:4]
        hs_actual = str(row['HS_Actual_from_validation'])[:4]

        # Check 2-digit accuracy
        if best_hs_code[:2] == hs_actual[:2]:
            correct_two_digits += 1

        # Check 4-digit accuracy
        if best_hs_code == hs_actual:
            correct_four_digits += 1

    accuracy_at_two_digits = correct_two_digits / total if total > 0 else 0
    accuracy_at_four_digits = correct_four_digits / total if total > 0 else 0

    return accuracy_at_two_digits, accuracy_at_four_digits

# Higher-level function to run multiple iterations (Monte Carlo simulation)
def montecarlo_simulation(hs_code_catalogue_df, validation_data_df, n_iterations):
    results = []

    # Preprocess the catalogue
    hs_code_catalogue_df = preprocess_catalogue(hs_code_catalogue_df)

    # Run the simulation n_iterations times
    for iteration in range(n_iterations):
        print(f"Running iteration {iteration + 1}/{n_iterations}...")
        # Run one iteration
        output_df = run_iteration(hs_code_catalogue_df, validation_data_df)

        # Calculate accuracy
        acc_two_digits, acc_four_digits = calculate_accuracy(output_df)

        # Store the results
        results.append([iteration + 1, acc_two_digits, acc_four_digits])

    # Convert results to DataFrame for further analysis
    results_df = pd.DataFrame(results, columns=['Iteration', 'Accuracy_At_Two_Digits', 'Accuracy_At_Four_Digits'])

    return results_df

# Example usage: Running Monte Carlo simulation
results_df = montecarlo_simulation(hs_code_catalogue_df, validation_data_df, n_iterations=50)
print(results_df)

# Generate box plot to visualize accuracy distribution
plt.figure(figsize=(10, 6))
results_df[['Accuracy_At_Two_Digits', 'Accuracy_At_Four_Digits']].boxplot()
plt.title('Monte Carlo Simulation: Accuracy at Two and Four Digits')
plt.ylabel('Accuracy')
plt.show()


Running iteration 1/50...


KeyboardInterrupt: 

In [20]:
results_df

,Iteration,Accuracy_At_Two_Digits,Accuracy_At_Four_Digits
0,1,0.25,0.20
1,2,0.15,0.05
2,3,0.05,0.00
3,4,0.15,0.00
4,5,0.15,0.05
5,6,0.15,0.15
6,7,0.15,0.05
7,8,0.05,0.05
8,9,0.15,0.00
9,10,0.05,0.05


In [15]:
import random
import os  # Optional, to safely handle paths

# Save the DataFrame to a CSV file with the randomized filename
output_file_path = '../MonteCarloSimulation_baseline.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


Results saved to ../MonteCarloSimulation_baseline.csv
